# 文本预处理



In [1]:
import collections
import re
from d2l import torch as d2l

将数据集读取到由多条文本行组成的列表中

In [3]:
d2l.DATA_HUB['time_machine'] = (d2l.DATA_URL + 'timemachine.txt',
                                '090b5e7e70c295757f55df93cb0a180b9691891a')

def read_time_machine():  
    """将时间机器数据集加载到文本行的列表中"""
    with open(d2l.download('time_machine'), 'r') as f:
        '''
        读取所有行：将文件内容读入一个列表，每个元素是一行字符串（包含换行符 \n）
        re.sub('[^A-Za-z]+', ' ', line):正则表达式替换,将所有非字母字符（至少一个）替换为单个空格。
        [^A-Za-z] ：匹配非英文大小写字母的字符。
        +：匹配一个或多个连续的非字母字符。
        目的：移除标点符号、数字、特殊符号，只保留单词字母。
        .strip():去除首尾空白,删除行首和行尾的空格、换行符等。
        .lower():转小写,将所有大写字母转为小写，实现 词汇归一化（如"The"和"the"视为同一词）。
        '''
        lines = f.readlines()
    return [re.sub('[^A-Za-z]+', ' ', line).strip().lower() for line in lines]

lines = read_time_machine() # 调用函数，返回清洗后的文本行列表
print(f'# 文本总行数: {len(lines)}')
print(lines[0])
print(lines[10])

# 文本总行数: 3221
the time machine by h g wells
twinkled and his usually pale face was flushed and animated the


每个文本序列又被拆分成一个词元列表

In [4]:
# lines: 字符串列表（每行一个句子/段落）
# token: 分词模式，默认为'word'（单词级），可选'char'（字符级）
def tokenize(lines, token='word'):  
    """将文本行拆分为单词或字符词元"""
    # line.split()：对每行字符串调用split()方法，按空白字符（空格、换行等）分割，返回单词列表。
    if token == 'word':
        return [line.split() for line in lines]
    # 字符模式：按字符分词：list(line)：将字符串直接转为字符列表，每个字符成为一个词元。
    elif token == 'char':
        return [list(line) for line in lines]
    else:
        print('错误：未知词元类型：' + token)

tokens = tokenize(lines)
for i in range(11):
    print(tokens[i])

['the', 'time', 'machine', 'by', 'h', 'g', 'wells']
[]
[]
[]
[]
['i']
[]
[]
['the', 'time', 'traveller', 'for', 'so', 'it', 'will', 'be', 'convenient', 'to', 'speak', 'of', 'him']
['was', 'expounding', 'a', 'recondite', 'matter', 'to', 'us', 'his', 'grey', 'eyes', 'shone', 'and']
['twinkled', 'and', 'his', 'usually', 'pale', 'face', 'was', 'flushed', 'and', 'animated', 'the']


构建一个字典，通常也叫做*词表*（vocabulary），
用来将字符串类型的词元映射到从$0$开始的数字索引中

为什么需要双向映射？
| 操作            | 使用场景      | 使用哪个映射              |
| ------------- | --------- | ------------------- |
| **编码**（词元→索引） | 将文本转为模型输入 | `self.token_to_idx` |
| **解码**（索引→词元） | 将模型输出转回文本 | `self.idx_to_token` |
| **查词频/未知词**   | 获取词元或索引信息 | 两者皆可                |


@property 的核心作用
- 语法糖：让方法调用看起来像属性访问（无括号）。
- 只读保护：外部可以读取值，但无法直接修改（除非提供 setter）。
- 封装性：隐藏内部实现细节，对外提供统一接口。

设计意图：
- 只读访问：外部可查看词频，但不能修改（避免破坏内部状态）。
- 封装保护：_token_freqs 是受保护属性（约定俗成，单下划线前缀），不应被外部直接访问。通过 property 提供受控的读取接口。

In [5]:
class Vocab:  
    """文本词表"""
    '''
    初始化词表，接收三个参数：
    tokens: 分词后的嵌套列表（如 [['the', 'time'], ['by', ...]]）
    min_freq=0: 词频阈值，低于该频率的词元被过滤
    reserved_tokens: 保留的特殊词元列表（如 ['<pad>', '<bos>']）
    '''
    def __init__(self, tokens=None, min_freq=0, reserved_tokens=None):
        # 处理默认参数，防止可变对象问题
        if tokens is None:
            tokens = []
        if reserved_tokens is None:
            reserved_tokens = []
        # 调用辅助函数统计词频，返回collections.Counter对象
        counter = count_corpus(tokens)
        '''
        按词频降序排列词元，格式为[(词元, 频率), ...]
        1. counter.items():将collections.Counter 对象转换为元素列表[(词元1,频率1),(词元2, 频率2),...]
        2. key=lambda x: x[1]:指定排序依据
        lambda x: x[1]表示按每个元组的第2个元素（即词频）排序。x[0]是词元字符串，x[1]是该词元的出现次数。
        3. reverse=True：降序排列，频率高的词元排在前面。
        默认reverse=False是升序，这里用True确保高频词优先被处理。
        4. sorted(...)：Python内置排序函数，返回全新列表，不修改原counter对象。
        '''
        self._token_freqs = sorted(counter.items(), key=lambda x: x[1],
                                   reverse=True)
        '''
        创建索引→词元的列表：['<unk>'] ：将未知词元放在索引0的位置，作为默认词元。
        + reserved_tokens：将保留词元（如['<pad>','<bos>','<eos>']）追加在后面。
        '''
        self.idx_to_token = ['<unk>'] + reserved_tokens
        '''
        通过字典推导式创建词元→索引的反向映射：
        enumerate(self.idx_to_token)：生成 (索引, 词元) 配对，如 (0, '<unk>'), (1, '<pad>')。
        推导式：对每个配对，构造 词元: 索引 的字典项。
        '''
        self.token_to_idx = {token: idx
                             for idx, token in enumerate(self.idx_to_token)}
        '''
        词表构建循环:按频率从高到低遍历词元;低于 min_freq 的词元跳过;新词元加入词表，索引动态分配
        顺序遍历：按之前排好的降序词频列表（从最高频到最低频）逐个处理词元。
        目的：确保高频词优先获得小索引，提升训练效率。
        '''
        for token, freq in self._token_freqs:
            # 频率过滤：一旦遇到词频低于min_freq的词元，立即终止循环。
            # 效率优化：因列表已排序，后续词元频率必然更低，无需继续检查。
            if freq < min_freq:
                break
            # 防重复检查：确保词元未被添加过（主要针对保留词元）。
            # 必要性：若reserved_tokens中包含已在文本出现的词（如 <pad>），跳过避免冲突。
            if token not in self.token_to_idx:
                # 追加到索引列表：将新词元添加到idx_to_token列表末尾。
                # 动态扩展：列表长度每增加1，新词元的索引就是当前最后一个位置。
                self.idx_to_token.append(token)
                '''
                分配索引：在字典中建立词元 →索引映射。
                索引计算：len(self.idx_to_token)-1获取刚追加词元的新索引。
                假设列表原为 ['<unk>', '<pad>', 'the']（长度=3）
                追加 'and' 后，长度=4，新索引=4-1=3
                结果：token_to_idx['and']=3
                '''
                self.token_to_idx[token] = len(self.idx_to_token) - 1
    # 返回词表大小，支持len(vocab)调用
    def __len__(self):
        return len(self.idx_to_token)
    '''
    词元→索引转换，支持单个词元或列表;未知词返回 <unk> 的索引（0）;递归处理列表
    '''
    def __getitem__(self, tokens):
        # 判断输入类型：检查 tokens 是否为单个词元（字符串）。
        # 条件：如果不是列表或元组，说明是单个词元，直接进入单元素处理分支。
        if not isinstance(tokens, (list, tuple)):
            # 查字典：在token_to_idx映射中查找词元。
            # 安全机制：.get()方法在词元不存在时返回self.unk（未知词索引，默认为0），避免KeyError。
            return self.token_to_idx.get(tokens, self.unk)
        '''
        批量处理：对词元列表/元组，使用列表推导式递归调用自己。
        递归：对tokens中每个token，执行self.__getitem__(token)（即回到第1步）。
        结果：返回索引列表。
        '''
        return [self.__getitem__(token) for token in tokens]
    # 索引→词元反向转换。注意：非法索引会抛出 IndexError
    def to_tokens(self, indices):
        # 判断输入类型 ：检查indices是否为 单个索引（整数）。
        # 条件：如果不是列表或元组，说明是单个整数索引，直接查询。
        if not isinstance(indices, (list, tuple)):
            '''
            直接索引：使用indices作为下标访问列表self.idx_to_token。
            结果：返回对应位置的词元字符串。
            风险：若索引越界（不在0~len(vocab)-1 范围内），会抛出IndexError。
            '''
            return self.idx_to_token[indices]
        # 批量处理：对索引列表/元组，使用列表推导式逐个转换
        return [self.idx_to_token[index] for index in indices]
    # 将<unk>索引定义为只读属性，固定返回0。
    @property
    def unk(self):
        return 0
    # 提供访问词频统计的接口。
    # 返回内部词频统计 _token_freqs（已排序的 [(词元, 频率), ...]）。
    @property
    def token_freqs(self):
        return self._token_freqs
'''
扁平化嵌套列表：[['a','b'], ['c']] → ['a','b','c']
使用collections.Counter统计词频
'''
def count_corpus(tokens):  
    """统计词元的频率"""
    if len(tokens) == 0 or isinstance(tokens[0], list):
        tokens = [token for line in tokens for token in line]
    return collections.Counter(tokens)

构建词表

In [6]:
'''
实例化词表：用之前分词得到的tokens（嵌套词元列表）创建Vocab对象。
1. 统计所有词元的频率（count_corpus）
2. 按频率降序排序
3. 初始化idx_to_token=['<unk>']
4. 遍历排序后的词元，依次添加到词表（索引从0开始递增）
5. 构建token_to_idx反向映射字典
'''
vocab = Vocab(tokens)
'''
获取字典视图：返回dict_items对象，包含所有 (词元, 索引) 键值对。
list(...):转为列表：将字典视图转换为可索引的列表，便于切片操作。结果：[('<unk>', 0), ('the', 1), ('time', 2), ...]
[:10]切片操作：取前10项，观察高频词及其索引分配情况。
print(...)打印结果：输出前10个词元-索引对，通常如下：
'''
print(list(vocab.token_to_idx.items())[:10])

[('<unk>', 0), ('the', 1), ('i', 2), ('and', 3), ('of', 4), ('a', 5), ('to', 6), ('was', 7), ('in', 8), ('that', 9)]


将每一条文本行转换成一个数字索引列表

In [7]:
for i in [0, 10]: # 循环遍历：只处理第0行和第10行文本（非连续的0到10）。
    print('文本:', tokens[i]) # 打印原始文本：显示tokens中第i行的词元列表。
    print('索引:', vocab[tokens[i]]) # 编码为索引：调用vocab的__getitem__方法，将词元列表转换为索引列表。

文本: ['the', 'time', 'machine', 'by', 'h', 'g', 'wells']
索引: [1, 19, 50, 40, 2183, 2184, 400]
文本: ['twinkled', 'and', 'his', 'usually', 'pale', 'face', 'was', 'flushed', 'and', 'animated', 'the']
索引: [2186, 3, 25, 1044, 362, 113, 7, 1421, 3, 1045, 1]


将所有功能打包到`load_corpus_time_machine`函数中

In [8]:
# 定义加载函数，max_tokens参数限制返回的词元数量（-1 表示不限制）。
def load_corpus_time_machine(max_tokens=-1):  
    """返回时光机器数据集的词元索引列表和词表"""
    # 读取原始文本行列表（已清洗为纯小写字符）。
    lines = read_time_machine()
    # 关键选择：使用字符级分词（而非单词级），将每行拆分为字符列表。
    tokens = tokenize(lines, 'char')
    # 基于字符构建词表，每个唯一字符分配一个索引。
    vocab = Vocab(tokens)
    '''
    双重列表推导：将嵌套的字符列表展平为一维索引序列。
    外层循环：for line in tokens遍历每一行（每个子列表），例如 line = ['t','h','e',' ',...]
    内层循环：for token in line对当前行，遍历每个字符词元，例如 token = 't', token = 'h'...
    表达式： vocab[token]调用 Vocab.__getitem__ 方法，将字符词元转换为数字索引。示例：'t' → 3, 'h' → 5...
    '''
    corpus = [vocab[token] for line in tokens for token in line]
    # 截断处理：若指定最大长度，只保留前max_tokens个字符索引。
    if max_tokens > 0:
        corpus = corpus[:max_tokens]
    # 返回语料库索引列表和词表对象，供后续语言模型使用
    return corpus, vocab
# len(corpus)：总字符数（约3万-4万）。
# len(vocab)：唯一字符数（英文文本约28-30个：字母+空格+标点）。
corpus, vocab = load_corpus_time_machine()
len(corpus), len(vocab)

(170580, 28)